# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to make sure that the team is continuously monitoring and analyzing the market data for AAPL in order to identify trends and predict market movements accurately. I should delegate tasks and ask questions to my team members to ensure we are on the right track.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Use statistical modeling and machine learning to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_u

Action Input: {"search_query": "AAPL user-defined risk tolerance criteria trading strategies"} 


Search results: Title: Apple Trading Strategies & Insights - LinkedIn
Link: https://www.linkedin.com/pulse/apple-trading-strategies-insights-quantifiedstrategies-apmpc
Snippet: Risk Tolerance is Important When Trading Apple Options​​ Trading options on Apple stock can be an effective way to manage risk because they allow ...
---
Title: AAPL Bullish Options Strategy Backtest Results
Link: https://optionalpha.com/blog/aapl-bullish-strategies-backtested
Snippet: Use Option Alpha to backtest three bullish strategies and compare the results, or search a database of AAPL options strategies. View risk ...
---
Title: Investing with options - Apple Inc - simple option buying strategies
Link: https://www.home.saxo/content/articles/options/investing-with-options---apple-inc---simple-option-buying-strategies-08092023
Snippet: Every investor or trader must conduct their own due diligence and take into 

I need to gather more information on the approved trading strategies for AAPL to provide the best analysis.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading-strategies-4689748"} 

Investing in US Markets
By clicking “Accept All Cookies”, you agree to the storing of cookies on your device to enhance site navigation, analyze site usage, and assist in our marketing efforts. Cookies Settings Accept All Cookies
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Derivatives
 Commodities
 Trading
 Automated Investing
 Brokers
 Fundamental Analysis
 Markets
 View All
 Simulator
 Login / Portfolio
 Trade
 Research
 My Games
 Leaderboard
 Banking
 Savings Accounts
 Certificates of Deposit (CDs)
 Money Market Accounts
 Checking Accounts
 View All
 Personal Finance
 Budgeting and Saving
 Personal Loans
 Insurance
 Mortgages
 Credit and Debt
 Student Loans
 Taxes
 Credit Cards
 Financial Literacy
 Retirement
 View All
 News
 Markets
 Compan

Final Answer: 
As per the analysis of the approved trading strategies for AAPL conducted by the Trading Strategy Developer, the detailed execution plans are as follows:

Long-Term Investment Strategy:
- Execution Method: Buy and hold AAPL stocks for an extended period, focusing on the company's growth potential and dividend payments.
- Timing: Purchase AAPL stocks when the market conditions are stable and hold for the long term.
- Trade Size: Allocate a portion of the portfolio to AAPL stocks based on the risk tolerance of the user.

Swing Trading Strategy:
- Execution Method: Take advantage of short- to medium-term price fluctuations in AAPL stocks by buying low and selling high.
- Timing: Monitor short-term market movements and execute trades accordingly.
- Trade Size: Adjust trade size based on the volatility of the market and potential price movements.

Options Trading Strategy:
- Execution Method: Utilize options contracts to speculate on the price movements of AAPL stocks.
- Timi

Thought: I now can give a great answer
Final Answer: Based on the search results from the internet, I have found various trading strategies for AAPL, including long-term investment, swing trading, options trading, and day trading. Each strategy comes with its own risks and potential rewards. To conduct a comprehensive risk analysis, I will need to delve deeper into each strategy, assess the market dynamics, and the current economic situation. I will also look into the historical performance of AAPL and potential risk factors such as market volatility, geopolitical events, and industry-specific risks. Once I have gathered all necessary information, I will provide detailed analysis and mitigation strategies to ensure that the proposed trading activities align with the firm's risk tolerance.

> Finished chain.
 

Based on the search results from the internet, I have found various trading strategies for AAPL, including long-term investment, swing trading, options trading, and day trading. 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Long-Term Investment Strategy:
- Potential Risks:
  - Market Volatility: Fluctuations in market conditions could impact the long-term growth potential of AAPL stocks.
  - Economic Downturn: A recession or economic crisis could negatively affect the company's performance and stock price.
- Mitigation Strategies:
  - Diversification: Allocate a portion of the portfolio to other asset classes to reduce reliance on AAPL stocks.
  - Regular Monitoring: Keep track of market conditions and adjust the investment strategy accordingly.

Swing Trading Strategy:
- Potential Risks:
  - Price Volatility: Rapid price movements in AAPL stocks could lead to unexpected losses.
  - Timing Risk: Incorrect timing of trades could result in missed opportunities or losses.
- Mitigation Strategies:
  - Technical Analysis: Use technical indicators to identify potential entry and exit points for trades.
  - Risk Management: Set stop-loss orders to limit losses and take profits at predetermined levels.

Options Trading Strategy:
- Potential Risks:
  - Options Expiry: Incorrect timing of options trades could result in the expiration of contracts.
  - Volatility Risk: Market volatility could impact the value of options contracts.
- Mitigation Strategies:
  - Options Hedging: Use options contracts to hedge against potential losses in AAPL stock.
  - Risk Assessment: Carefully analyze options contracts based on risk tolerance and market conditions.

Day Trading Strategy:
- Potential Risks:
  - Intraday Volatility: Rapid price movements within the trading day could lead to unexpected losses.
  - Margin Calls: Trading on margin could result in margin calls if trades go against expectations.
- Mitigation Strategies:
  - Stop-Loss Orders: Set stop-loss orders to limit losses and protect capital.
  - Margin Management: Monitor margin levels and avoid excessive leverage in day trading activities.

Overall, it is essential to consider the specific risks associated with each trading strategy for AAPL and implement appropriate mitigation strategies to protect the firm's capital and optimize returns. By carefully analyzing market dynamics, monitoring risk factors, and adjusting trading strategies accordingly, the team can effectively manage the execution of trades for AAPL in a way that aligns with the firm's risk tolerance and investment objectives.